## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df['text'].head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    Ever get the feeling your life circles the rou...
2    Why the Truth Might Get You Fired October 29, ...
3    Videos 15 Civilians Killed In Single US Airstr...
4    Print \nAn Iranian woman has been sentenced to...
Name: text, dtype: object

In [5]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
###Drop Nan Values
df=df.dropna() #we can not replace text data


In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
X.shape

(18285, 4)

In [13]:
y.shape

(18285,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.18.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [31]:
messages=X.copy()

In [32]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [33]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [40]:
messages.reset_index(inplace=True)

In [41]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [42]:
import nltk
import re
from nltk.corpus import stopwords

In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [45]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1256, 88, 4011, 4958, 2734, 1518, 4362, 3432, 2255, 1782],
 [719, 2218, 997, 4625, 4773, 2399, 1945],
 [3526, 1803, 4258, 3825],
 [2167, 2633, 3709, 3448, 115, 948],
 [1171, 4773, 3243, 203, 4311, 1439, 4773, 1087, 2151, 3160],
 [3089,
  2988,
  1665,
  3971,
  807,
  4891,
  2305,
  2180,
  1665,
  193,
  3049,
  3616,
  4713,
  886,
  1945],
 [4751, 1937, 3278, 364, 3456, 92, 3826, 2089, 2483, 1783, 4705],
 [2845, 1824, 3955, 4131, 2470, 1766, 4891, 340, 2483, 1783, 4705],
 [4966, 1341, 3101, 2102, 4545, 1258, 4789, 1967, 4891, 4136],
 [4934, 354, 2240, 3420, 900, 1914, 1556, 1486],
 [226, 1890, 1, 1379, 382, 576, 2458, 4534, 1958, 3735, 1948],
 [3448, 2688, 2734, 1258, 4891, 2470],
 [3510, 3191, 1551, 4740, 2374, 1033, 790, 3362, 2529],
 [689, 397, 4293, 4860, 1688, 1073, 3448, 2483, 1783, 4705],
 [4734, 3495, 2817, 642, 3581, 2483, 1783, 4705],
 [4531, 3055, 1415, 380, 450, 237, 4101, 46, 3126, 68],
 [1040, 662, 2218],
 [4080, 29, 3467, 686, 4891, 1187, 4153, 1945],
 [3742, 3080,

In [48]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [49]:
onehot_repr[1]

[719, 2218, 997, 4625, 4773, 2399, 1945]

### Embedding Representation

In [50]:
max_length = max(len(sentence) for sentence in corpus)
print("Maximum sentence length:", max_length)

Maximum sentence length: 299


In [51]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3432 2255 1782]
 [   0    0    0 ... 4773 2399 1945]
 [   0    0    0 ... 1803 4258 3825]
 ...
 [   0    0    0 ... 2483 1783 4705]
 [   0    0    0 ... 4856  104 2704]
 [   0    0    0 ... 1426 3833  537]]


In [52]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,  719, 2218,  997, 4625, 4773, 2399, 1945])

In [53]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1256,
         88, 4011, 4958, 2734, 1518, 4362, 3432, 2255, 1782])

In [64]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None, sent_length))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_10 (LSTM)                  │ (None, 100)            │        56,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 256,501 (1001.96 KB)

 Trainable params: 256,501 (1001.96 KB)

 Non-trainable params: 0 (0.00 B)

None


In [65]:
len(embedded_docs),y.shape

(18285, (18285,))

In [66]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [67]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [86]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7806 - loss: 0.4421 - val_accuracy: 0.9167 - val_loss: 0.1982
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9366 - loss: 0.1577 - val_accuracy: 0.9206 - val_loss: 0.1920
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9550 - loss: 0.1175 - val_accuracy: 0.9180 - val_loss: 0.2040
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9699 - loss: 0.0823 - val_accuracy: 0.9150 - val_loss: 0.2358
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9769 - loss: 0.0651 - val_accuracy: 0.9085 - val_loss: 0.2317
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9796 - loss: 0.0631 - val_accuracy: 0.9118 - val_loss: 0.3160
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9846 - loss: 0.0442 - val_accuracy: 0.9109 - val_loss: 0.3086
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9871 - loss: 0.0391 - val_accu

### Adding Dropout

In [78]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3)) #30% of this LSTM while training data will be disable
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [87]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [88]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [89]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [90]:
from sklearn.metrics import confusion_matrix

In [91]:
confusion_matrix(y_test,y_pred)

array([[3117,  302],
       [ 252, 2364]], dtype=int64)

In [92]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9082021541010771

In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

